### BLS API retrieval tool

In [1]:
# Import preliminaries
import requests
import pandas as pd
import json

# Local file with API key
import config


def api(series, date_range):
    """Collect list of series from BLS API for given dates"""
    # The url for BLS API v2
    url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

    # API key in config.py which contains: bls_key = 'key'
    key = '?registrationkey={}'.format(config.bls_key)

    # Handle dates
    dates = [(str(date_range[0]), str(date_range[1]))]
    while int(dates[-1][1]) - int(dates[-1][0]) > 10:
        dates = [(str(date_range[0]), str(date_range[0] + 9))]
        d1 = int(dates[-1][0])
        while int(dates[-1][1]) < date_range[1]:
            d1 = d1 + 10
            d2 = min([date_range[1], d1 + 9])
            dates.append((str(d1), (d2)))

    df = pd.DataFrame()

    for start, end in dates:
        # Submit the list of series as data
        data = json.dumps({
            "seriesid": list(series.keys()),
            "startyear": start, "endyear": end})

        # Post request for the data
        p = requests.post(
            '{}{}'.format(url, key),
            headers={'Content-type': 'application/json'},
            data=data).json()
        for s in p['Results']['series']:
            col = series[s['seriesID']]
            for r in s['data']:
                date = pd.to_datetime('{} {}'.format(
                    r['periodName'], r['year']))
                df.at[date, col] = float(r['value'])
    df = df.sort_index()
    # Output results
    print('Request Status: {}'.format(p['status']))
    print(f'Columns: {df.shape[1]}, Rows: {df.shape[0]}')
    print(f'Latest date: {df.index[-1].date()}')
    
    return df

In [4]:
# Series stored as {id: name}
slist = {'CUUR0000SA0': 'CPI'}

# Start year and end year
dates = (1994, 2018)

In [5]:
df = api(slist, dates)

Request Status: REQUEST_SUCCEEDED
Columns: 1, Rows: 295
Latest date: 2018-07-01


,CPI
1994-01-01,8.702174
1994-02-01,8.731935
1994-03-01,8.761696
1994-04-01,8.773601
1994-05-01,8.779553
1994-06-01,8.809314
1994-07-01,8.833123
1994-08-01,8.868836
1994-09-01,8.892645
1994-10-01,8.898598


In [6]:
df['fifteen_bucks'] = df.loc['']

,CPI
1994-01-01,146.200
1994-02-01,146.700
1994-03-01,147.200
1994-04-01,147.400
1994-05-01,147.500
1994-06-01,148.000
1994-07-01,148.400
1994-08-01,149.000
1994-09-01,149.400
1994-10-01,149.500
